In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# initalize file paths
numtracts_file = "/Users/armanimanov/Downloads/number of tracts.csv"
meanlength_file = "/Users/armanimanov/Downloads/mean length(mm).csv"
totalvol_file = "/Users/armanimanov/Downloads/total volume(mm^3).csv"
demographic_file = "/Users/armanimanov/Downloads/abcd_p_demo.csv"

In [3]:
numtracts_metrics = pd.read_csv(numtracts_file)
meanlength_metrics = pd.read_csv(meanlength_file)
totalvol_metrics = pd.read_csv(totalvol_file)
demographic_metrics = pd.read_csv(demographic_file)

numtracts_metrics['src_subject_id'] = numtracts_metrics['src_subject_id'].str.replace('sub-', '')
meanlength_metrics['src_subject_id'] = meanlength_metrics['src_subject_id'].str.replace('sub-', '')
totalvol_metrics['src_subject_id'] = totalvol_metrics['src_subject_id'].str.replace('sub-', '')
demographic_metrics['src_subject_id'] = demographic_metrics['src_subject_id'].str.replace('NDAR_', 'NDAR')

demographic_metrics = demographic_metrics[demographic_metrics['demo_sex_v2'] != 3.0]
demographic_metrics = demographic_metrics[demographic_metrics['eventname'] == 'baseline_year_1_arm_1']

numtracts_merge = pd.merge(numtracts_metrics, demographic_metrics, on='src_subject_id')
meanlength_merge = pd.merge(meanlength_metrics, demographic_metrics, on='src_subject_id')
totalvol_merge = pd.merge(totalvol_metrics, demographic_metrics, on='src_subject_id')

### Modifications for proposal, May 1 2024

In [28]:
tracts = [col for col in meanlength_merge.columns if col in ['Association_CingulumR_Parahippocampal', 'Association_InferiorFrontoOccipitalFasciculusL',
                                                                    'Association_MiddleLongitudinalFasciculusL', 'Association_SuperiorLongitudinalFasciculusL_2']]  # Adjust prefix as needed


In [114]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
from scipy.stats import ttest_ind
import plotly.io as pio

female_color = 'rgba(229,134,6, 0.7)'
male_color = 'rgba(29, 90, 120, 0.7)'

fig = make_subplots(rows=1, cols=1)
t_stats = []

for i, tract in enumerate(tracts):
    female_data = meanlength_merge[meanlength_merge['demo_sex_v2'] == 2][tract]
    male_data = meanlength_merge[meanlength_merge['demo_sex_v2'] == 1][tract]
    
    t_stat, p_value = ttest_ind(female_data, male_data, equal_var=True, alternative='two-sided', nan_policy='omit')
    t_stats.append((t_stat, p_value))

    # FEMALE
    fig.add_trace(go.Box(
        x=[i - 0.2] * len(female_data),
        y=female_data,
        name='F',
        marker_color=female_color,
        fillcolor=female_color,
        line=dict(color='rgb(50,50,50)'),
        boxpoints='outliers',
        jitter=0.1,
        pointpos=0,
        width=0.35,
        showlegend=True if i == 0 else False
    ), row=1, col=1)

    # MALE
    fig.add_trace(go.Box(
        x=[i + 0.2] * len(male_data),
        y=male_data,
        name='M',
        marker_color=male_color,
        fillcolor=male_color,
        line=dict(color='rgb(0,0,0)'),
        boxpoints='outliers',
        jitter=0.1,
        pointpos=0,
        width=0.35,
        showlegend=True if i == 0 else False
    ), row=1, col=1)


offset = 10

for idx, (t_stat, p_value) in enumerate(t_stats):
    max_y_value = max(max(meanlength_merge[meanlength_merge['demo_sex_v2'] == 2][tracts[idx]]),
                      max(meanlength_merge[meanlength_merge['demo_sex_v2'] == 1][tracts[idx]]))
    fig.add_annotation(x=idx, y=max_y_value + offset,
                   text=f'T = {t_stat:.2f}',
                   showarrow=False,
                   font=dict(color="black", size=12))

fig.update_layout(
    xaxis=dict(
        title_font=dict(size=20),
        tickfont=dict(size=16, family="Arial"),
        tickmode='array',
        tickvals=list(range(len(tracts))),
        ticktext=short_tracts
    ),
    yaxis=dict(
        title="Mean Length (mm)",
        title_font=dict(size=20),
        gridcolor='lightgray',
        gridwidth=1,
        zerolinecolor='lightgray',
        tickfont=dict(size=20),
        tickvals=[0, 50, 100, 150, 200],
        ticktext=['0', '50', '100', '150', '200']
    ),
    plot_bgcolor='white',
    boxmode='group',
    boxgroupgap=0.5,
    height=550,
    width=550,
    margin=dict(l=50, r=50, t=50, b=50),
    legend=dict(
        title='', 
        orientation='h', 
        yanchor='bottom', 
        y=1.02, 
        xanchor='right', 
        x=1, 
        font=dict(size=18, family='Arial Black')
    ),
)

fig.show()

pio.write_image(fig, '/Users/armanimanov/Desktop/neurosurg/baselineplot.svg', format='svg')


In [112]:
t_stats

[(-14.625022101095082, 6.280233213551534e-48),
 (-39.90546789833624, 0.0),
 (-11.5888691450926, 7.488207844576264e-31),
 (-13.332877759276403, 3.3606193551628123e-40)]